<a href="https://colab.research.google.com/github/Leonardpepa/High-Performance-Computing/blob/main/Leonard_Pepa_ics20033_hpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [11]:
!base64 /dev/urandom | head -c 1000000000 > file.txt

char_freq  char_freq_gpu.cu  sample_data


In [145]:
# Character frequency
!nvcc char_freq_gpu.cu -O2 -o char_freq

# 256 threads per block
!./char_freq file.txt 256

# 512 threads per block
!./char_freq file.txt 512

# 1024 threads per block
!./char_freq file.txt 1024

file size is 1000000000
0 = 0
1 = 0
2 = 0
3 = 0
4 = 0
5 = 0
6 = 0
7 = 0
8 = 0
9 = 0
10 = 12987012
11 = 0
12 = 0
13 = 0
14 = 0
15 = 0
16 = 0
17 = 0
18 = 0
19 = 0
20 = 0
21 = 0
22 = 0
23 = 0
24 = 0
25 = 0
26 = 0
27 = 0
28 = 0
29 = 0
30 = 0
31 = 0
32 = 0
33 = 0
34 = 0
35 = 0
36 = 0
37 = 0
38 = 0
39 = 0
40 = 0
41 = 0
42 = 0
43 = 15423983
44 = 0
45 = 0
46 = 0
47 = 15430208
48 = 15433351
49 = 15425159
50 = 15412873
51 = 15416629
52 = 15423031
53 = 15423249
54 = 15420399
55 = 15420610
56 = 15424846
57 = 15417030
58 = 0
59 = 0
60 = 0
61 = 0
62 = 0
63 = 0
64 = 0
65 = 15414167
66 = 15424655
67 = 15418680
68 = 15417905
69 = 15424965
70 = 15422413
71 = 15415026
72 = 15423657
73 = 15419608
74 = 15420981
75 = 15425742
76 = 15418745
77 = 15428191
78 = 15423513
79 = 15425146
80 = 15415291
81 = 15426071
82 = 15426459
83 = 15417168
84 = 15424178
85 = 15425256
86 = 15432981
87 = 15415049
88 = 15423954
89 = 15425035
90 = 15424400
91 = 0
92 = 0
93 = 0
94 = 0
95 = 0
96 = 0
97 = 15424578
98 = 15424164
99 = 1

In [149]:
# String matching
!nvcc string_matching_gpu.cu -O2 -o string_matching

# 256 threads per block
!./string_matching E.coli attgatgg 256

# 512 threads per block
!./string_matching E.coli attgatgg 512

# 1024 threads per block
!./string_matching E.coli attgatgg 1024



File size: 4638690, blocks: 18120, threads per block: 256, total_threads: 4638720
Total time (seconds): 0.009825
Kernel time (seconds): 0.000246
Data transfer time(seconds): 0.009579

Total matches = 163


File size: 4638690, blocks: 9060, threads per block: 512, total_threads: 4638720
Total time (seconds): 0.009649
Kernel time (seconds): 0.000251
Data transfer time(seconds): 0.009398

Total matches = 163


File size: 4638690, blocks: 4530, threads per block: 1024, total_threads: 4638720
Total time (seconds): 0.010357
Kernel time (seconds): 0.000273
Data transfer time(seconds): 0.010084

Total matches = 163



In [153]:
# Count Sort
!nvcc count_sort_gpu.cu -O2 -o count_sort

# 256 threads per block
!./count_sort 900000 256

# 512 threads per block
!./count_sort 900000 512

# 1024 threads per block
!./count_sort 900000 1024




Array size: 900000, blocks: 3516, threads per block: 256, total_threads: 900096
Total time (seconds): 1.594422
Kernel time (seconds): 1.590626
Data transfer time (seconds): 0.003797
Array is sorted!

Array size: 900000, blocks: 1758, threads per block: 512, total_threads: 900096
Total time (seconds): 1.425085
Kernel time (seconds): 1.421342
Data transfer time (seconds): 0.003742
Array is sorted!

Array size: 900000, blocks: 879, threads per block: 1024, total_threads: 900096
Total time (seconds): 1.442563
Kernel time (seconds): 1.438861
Data transfer time (seconds): 0.003702
Array is sorted!
